# Import Libs

In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

from sklearn.preprocessing import FunctionTransformer, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, train_test_split

from lightgbm import LGBMRegressor
import lightgbm as lgb

# Import Data

In [60]:
data_path = 'C:/Users/ticom/VLabs/Data'
df = pd.read_csv(data_path+'/sales_20_21_train.csv', header=0, parse_dates=['DT_VENDA'])
df_sub = pd.read_csv(data_path+'/sample_submission.csv',header=0)

In [61]:
df_sub.head()

,ID_CLIENTE,VALOR
0,4,0.0
1,9,0.0
2,12,0.0
3,15,0.0
4,19,0.0


In [271]:
df['ID_CLIENTE'].nunique()

210269

# Estratégia

Temos dados de janeiro de 2020 até o fim de fevereiro de 2021 e precisamos prever o quanto cada cliente gastará nos próximos 90 dias. Vamos treinar o modelo com os dados de jan20 até nov20 e usaremos os valores de dez20, jan21 e fev21 como target do treinamento.

A submissão será feita com os dados de abr20 até fim de fev21.

In [5]:
# intervalo de 90 dias
nov_d = pd.to_timedelta(90,unit='d')

Definindo início do período de validação (target do treino):

In [6]:
val_start = '2020-11-25'

In [7]:
df_train = df[df['DT_VENDA']<=val_start]
df_test = df[df['DT_VENDA']>'2020-04-01']

In [414]:
df_train

,ID_VENDA,DT_VENDA,LOJA,QTD_SKU,VALOR,ID_CLIENTE,CANAL
0,1176550,2020-01-02,37,1,9.50,337763,FIS
1,11683461,2020-01-02,15,1,6.64,195687,FIS
2,3326484,2020-01-02,29,5,446.87,56059,FIS
3,7317475,2020-01-02,18,1,6.13,297533,FIS
4,4366938,2020-01-02,23,1,13.17,181284,FIS
...,...,...,...,...,...,...,...
1224386,3147575,2020-11-25,34,1,13.56,102169,FIS
1224387,3148349,2020-11-25,34,2,28.04,32611,FIS
1224388,11661717,2020-11-25,13,8,168.82,356427,FIS
1224389,3148119,2020-11-25,34,1,3.97,229386,FIS


In [210]:
df_test

,ID_VENDA,DT_VENDA,LOJA,QTD_SKU,VALOR,ID_CLIENTE,CANAL
355932,4336789,2020-04-02,24,2,17.48,241846,FIS
355933,96480,2020-04-02,31,7,106.75,319966,ECM
355934,4181455,2020-04-02,27,4,260.25,295528,FIS
355935,4336979,2020-04-02,24,6,93.78,125463,FIS
355936,6582700,2020-04-02,3,3,113.04,148168,FIS
...,...,...,...,...,...,...,...
1630607,2479951,2021-02-24,38,3,94.69,23473,FIS
1630608,10228861,2021-02-24,2,5,141.58,259864,FIS
1630609,11911135,2021-02-24,13,12,191.66,299242,FIS
1630610,11202897,2021-02-24,18,3,71.57,353139,IFOOD


# Functions

In [8]:
primary_key = ['ID_VENDA']
cat_vars = ['LOJA','ID_CLIENTE','CANAL']
num_vars = ['QTD_SKU']
time_vars = ['DT_VENDA']
target = ['VALOR']

In [9]:
max_date = df['DT_VENDA'].max()
min_date = df['DT_VENDA'].min()
print('max date: ',max_date,'\nmin date: ',min_date,'\nval date: ',val_start)

max date:  2021-02-24 00:00:00 
min date:  2020-01-02 00:00:00 
val date:  2020-11-25


In [10]:
def fix_dtype_func(x):
    """ Acerta os tipos das colunas """
    x=x.copy()
    try:
        for var in cat_vars:
            x[var] = x[var].astype('O')
        for var in num_vars:
            x[var] = x[var].astype('int')
        #for var in time_vars:
        #    x[var] = pd.to_datetime(x[var])
    except Exception as err:
        print('Erro em fix_dtype_func')
        print(err)
    return x

# Viabiliza usar a função como um transformador do scikit-learn
get_fix_dtype = FunctionTransformer(fix_dtype_func)

def time_features_func(x):
    """ Cria colunas com ano, mês e dia das vendas"""
    # cria cópia para não alterar o df original por acidente
    x=x.copy()
    try:
        x['ANO'] = x['DT_VENDA'].dt.year
        x['MES'] = x['DT_VENDA'].dt.month
        x['DIA'] = x['DT_VENDA'].dt.day
        #x = x.drop(['DT_VENDA'],axis=1).copy()
        #x = x.set_index('DT_VENDA')
    except Exception as err:
        print('Erro em fix_dtype_func')
        print(err)
    return x

# Viabiliza usar a função como um transformador do scikit-learn
get_time_features = FunctionTransformer(time_features_func)

# Preprocessing

In [11]:
def treated_data(df_pp):
    max_date = df_pp['DT_VENDA'].max()
    
    #agrupar por cliente somando o valor
    cust_revenue = df_pp.groupby(['ID_CLIENTE'])['VALOR'].sum().copy()
    #ordenar por data
    cust_revenue.sort_values(ascending=False, inplace=True)
    # transformar em df
    cust_rev = pd.DataFrame(cust_revenue)
    # feature com número de compras feitas no período de teste
    cust_rev['n_compras'] = df_pp.groupby(['ID_CLIENTE'])['ID_VENDA'].count()
    # std entre os valores das compras de cada cliente
    cust_rev['std entre compras'] = df_pp.groupby('ID_CLIENTE')['VALOR'].std()
    # preenche NAN com 0
    #cust_rev['std entre compras'] = cust_rev['std entre compras'].fillna(0)
    # meadia entre os valores das compras de cada cliente
    cust_rev['valor medio'] = df_pp.groupby('ID_CLIENTE')['VALOR'].mean()
    
    # diferença entre a data da primeira e ultima compra
    cust_rev['periodo_compras'] = df_pp.groupby(['ID_CLIENTE'])['DT_VENDA'].max() - df_pp.groupby(['ID_CLIENTE'])['DT_VENDA'].min()
    cust_rev['periodo_compras'] = cust_rev['periodo_compras'].apply(lambda x: x.days)
    # data da ultima compra
    cust_rev['ultima compra'] = max_date - df_pp.groupby(['ID_CLIENTE'])['DT_VENDA'].max()
    cust_rev['ultima compra'] = cust_rev['ultima compra'].apply(lambda x: x.days)
    # soma do VALOR dos ultimos 3 meses de cada cliente
    cust_rev['valor_3m'] = df_pp[
            df_pp['DT_VENDA']>(df_pp['DT_VENDA'].max() -pd.to_timedelta(90, unit='d'))
            ].groupby(['ID_CLIENTE'])['VALOR'].sum()
    
    return cust_rev.fillna(0)

In [12]:
x_train = treated_data(df_train)
x_test = treated_data(df_test)

In [13]:
x_train.head()

,VALOR,n_compras,std entre compras,valor medio,periodo_compras,ultima compra,valor_3m
ID_CLIENTE,,,,,,,
104935,97612.79,50,2759.341941,1952.255800,284,8,19086.18
377218,90207.54,48,933.892397,1879.323750,322,1,22516.99
260921,50840.31,107,166.083836,475.143084,299,27,18023.97
99880,39855.66,116,669.093693,343.583276,323,0,4532.59
13657,36832.62,55,484.759710,669.684000,328,0,14814.25


In [416]:
x_test.head()

,VALOR,n_compras,std entre compras,valor medio,periodo_compras,ultima compra,valor_3m
ID_CLIENTE,,,,,,,
377218,95759.93,50,920.877760,1915.198600,324,2,19755.91
104935,87665.94,50,2583.622370,1753.318800,228,75,9516.43
13657,43870.65,57,529.093980,769.660526,314,14,11204.93
99880,43493.29,92,713.235719,472.753152,320,6,13508.67
260921,40232.65,78,161.455085,515.803205,209,118,0.00


## Target

In [14]:
# cria uma lista vazia com o os clientes no index 
y_aux = pd.Series(np.zeros(len(x_train)))
y_aux.index = x_train.index

In [15]:
# cria o target para o treino como a soma da var "VALOR" agrupada por cliente 
# entre as datas de 25 de nov20 a 24 de fev21
y_train = df[df['DT_VENDA']>val_start].groupby(['ID_CLIENTE'])['VALOR'].sum()
y_train = (pd.DataFrame(y_aux).join(y_train)).fillna(0).drop(0,axis=1)

In [21]:
 y_train

,VALOR
ID_CLIENTE,
104935,9516.43
377218,19755.91
260921,0.00
99880,13508.67
13657,11279.34
...,...
379328,0.00
137298,260.80
221081,0.00


In [22]:
x_train.shape

(209144, 7)

## Split de dados - treino e teste

In [91]:
x_train1, x_test1, y_train1, y_test1= train_test_split(x_train, y_train, test_size=0.3, random_state=0)

# LGBM Model

In [18]:
lgbm = LGBMRegressor(random_state=0)

In [92]:
lgbm.fit(x_train1,y_train1)
pred_val = lgbm.predict(x_test1)
score = mean_squared_error(y_test1,pred_val, squared=False)
score

253.79728462850298

Valildação cruzada:

In [109]:
score_cross = cross_val_score(lgbm, X=x_train, y=y_train, cv=5, scoring='neg_root_mean_squared_error')

In [111]:
score_cross.mean()

-242.94940497387006

# Predição

In [39]:
lgbm.fit(x_train,y_train)
pred = lgbm.predict(x_test)

In [40]:
pred_serie = pd.DataFrame(pred, index=x_test.index, columns=['VALOR'])
pred_serie

,VALOR
ID_CLIENTE,
377218,9669.820780
104935,4682.707645
13657,6881.946747
99880,8013.421372
260921,1554.973266
...,...
152754,27.369933
153097,33.306277
254282,27.369933


## Preparar o arquivo de submissão:

In [62]:
sub_aux = df_sub['ID_CLIENTE']

In [63]:
pred1 = pd.DataFrame(sub_aux).join(pred_serie).fillna(0)
pred1

,ID_CLIENTE,VALOR
0,4,178.160167
1,9,0.000000
2,12,170.419915
3,15,28.193755
4,19,411.847361
...,...,...
126611,384409,0.000000
126612,384411,0.000000
126613,384415,217.374446
126614,384418,0.000000


In [65]:
pred1.to_csv(r"C:/Users/ticom/VLabs/Submissions/sub1-2.csv", index=False)